# Prepare the dataset
The dataset available in the following links \\
The base dataset (Non-noise): https://drive.google.com/file/d/1OH5vczwBYIxXXpX2VaYWmLXRUvJBBxQN/view?usp=sharing \\
The noise-adding dataset: https://drive.google.com/file/d/1QSA5HmpNdtrJzeGUFfv6MQVWR0AaHAz9/view?usp=sharing \\

Note that the labels are transformed into YOLOv5 format and divide into training and test sets so that it is not necessary to perform preprocessing.

In [ ]:
#Unzip the dataset
#Run only this cell

!unzip noise_dataset.zip -d /content
!unzip /content/drive/MyDrive/Yolo_1.zip -d /content

## The following cells contain code to perform transformation and preprocessing images

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob as g
import cv2
from tqdm.notebook import tqdm
from shutil import copy, move
from google.colab.files import download
import pandas as pd

In [ ]:
new_img_train = '/content/newDataset/images/train'
new_img_val = '/content/newDataset/images/val'
new_lbl_train = '/content/newDataset/label/train'
new_lbl_val = '/content/newDataset/label/val'

os.makedirs(new_img_train, exist_ok=True)
os.makedirs(new_img_val, exist_ok=True)
os.makedirs(new_lbl_train, exist_ok=True)
os.makedirs(new_lbl_val, exist_ok=True)


In [ ]:
#Show an image with its bounding box

from PIL import Image, ImageDraw

#Show an image with bounding box
path = '/content/images'
df = pd.read_csv('/content/faces.csv')
image_path = str(df['image_name'].iloc[0])
x0 = int(df['x0'].iloc[0])
y0 = int(df['y0'].iloc[0])
x1 = int(df['x1'].iloc[0])
y1 = int(df['y1'].iloc[0])
image = Image.open(os.path.join(path,image_path))
draw = ImageDraw.Draw(image)

draw.rectangle([(x0,y0), (x1,y1)], outline='red', width=2)
image.show()

In [ ]:
#Convert from csvfile to label and divide images to train/test folders

df = pd.read_csv('/content/faces.csv')
nums = len(df)
train_img_path = '/content/newDataset/images/train'
val_img_path = '/content/newDataset/images/val'
train_lbl_path = '/content/newDataset/label/train'
val_lbl_path = '/content/newDataset/label/val'
split_ratio = 0.85
for index in tqdm(range(len(df))):
  image_path = str(df['image_name'].iloc[index])
  width = int(df['width'].iloc[index])
  height = int(df['height'].iloc[index])
  x0 = int(df['x0'].iloc[index])
  y0 = int(df['y0'].iloc[index])
  x1 = int(df['x1'].iloc[index])
  y1 = int(df['y1'].iloc[index])
  #normalized
  normalized_x0 = x0 / width
  normalized_y0 = y0 / height
  normalized_x1 = x1 / width
  normalized_y1 = y1 / height

  #convertToYolo
  x_center = (normalized_x0 + normalized_x1) / 2
  y_center = (normalized_y0 + normalized_y1) / 2
  bbox_width = normalized_x1 - normalized_x0
  bbox_height = normalized_y1 - normalized_y0

  if index < nums * split_ratio:
    label_path = os.path.join(train_lbl_path, image_path.split('.')[0] + '.txt')
    f = open(label_path, 'w')
    yolo_line = f'{0} {x_center} {y_center} {bbox_width} {bbox_height}'
    f.write(yolo_line)
    f.close()
    copy(os.path.join('/content/images', image_path), train_img_path)
  else:
    label_path = os.path.join(val_lbl_path, image_path.split('.')[0]+ '.txt')
    f = open(label_path, 'w')
    yolo_line = f'{0} {x_center} {y_center} {bbox_width} {bbox_height}'
    f.write(yolo_line)
    f.close()
    copy(os.path.join('/content/images', image_path), val_img_path)

In [ ]:
print(len(os.listdir('/content/newDataset/images/train')))
print(len(os.listdir('/content/newDataset/images/val')))

In [ ]:
# plot a grid of images
names = g('/content/newDataset/label/*/*')
print(names)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

images = []
for _ in range(25):
    n = np.random.randint(0, len(names))
    f = open(names[n])

    lines = f.readlines()
    classes = list(map(lambda x: int(x[0]), lines))
    lines = list(map(lambda x:x.rstrip()[2:], lines))
    objects = list(map(lambda x:(x.split()), lines))

    img = cv2.imread(names[n].replace('txt','jpg').replace('label', 'images'))
    print(type(img))
    for c, bbox in zip(classes, objects):
        bbox = list(map(lambda x:float(x), bbox))
        x,y,w,h = bbox
        img_h = img.shape[0]
        img_w = img.shape[1]
        x = int(x * img_w)
        w = int(w * img_w)
        y = int(y * img_h)
        h = int(h * img_h)
        color = (255,100,50)
        cv2.rectangle(img , (int(x-w/2), int(y-h/2)), (int(x+w/2), int(y+h/2)), color , 6)
    # plt.figure(figsize = (8,8))
    # plt.imshow(img[:,:,::-1]); plt.axis('off')
    # print(f'number of bounding boxes : {len(classes)}')
    images.append(img[:,:,::-1])
fig = plt.figure(figsize=(16., 16.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(5 ,5),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )

for ax, im in zip(grid, images):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)
    ax.axis('off')

plt.show()

In [ ]:
labels_address = '/content/newDataset/labels/train/'
imgs_address = '/content/newDataset/images/train/'
np.random.seed(101)
names = os.listdir(imgs_address)
randvec = np.random.rand(len(names))
i = 0
for name in tqdm(names[:]):
  i += 1
  copy(imgs_address + name,  '/content/Yolo_1/images/train/' + name)
  copy(labels_address + name.split('.')[0] + '.txt' , '/content/Yolo_1/labels/train/' + name.split('.')[0] + '.txt')

# Run Model

In [ ]:
import torch
from IPython.display import clear_output
torch.cuda.get_device_name()

In [ ]:
#clone the model and write custom dataset

!git clone https://github.com/ultralytics/yolov5.git
!pip install -qr /content/yolov5/requirements.txt
%cd yolov5
clear_output()
f = open('/content/yolov5/data/dataset.yaml', 'w')
f.write('train: /content/content/noise_dataset/images/train')
f.write('\nval: /content/content/noise_dataset/images/val')
f.write('\nnc: {}'.format(1))
f.write("\nnames: ['Face']")

f.close()

In [ ]:
#train the model from scratch
!python train.py --multi-scale --batch 32 --data /content/yolov5/data/dataset.yaml --cfg /content/yolov5/models/yolov5s.yaml --epochs num_epoch --weights path/to/weights


In [ ]:
#Run test
!python val.py --batch 32 --data /content/yolov5/data/dataset.yaml --weights path/to/weights

In [ ]:
#Inference mode
!python detect.py --source path/to/source --weights path/to/weights


You can find the weights here: \\
For non-noise dataset weight: https://drive.google.com/file/d/1b10veNEl4LDnjsaODdsa90D2r4ztoT9h/view?usp=sharing \\
For noise dataset weight: https://drive.google.com/file/d/1FgVozWlrhPHoPQ47pW4cUXEBnezU7QzL/view?usp=sharing \\
or just train from scrach